# Day 1 - Exercise 3 - Sentiment Analysis

## Necessary imports

In order to handle the data properly we have to import the data and the modules we need:

In [1]:
# modules
import pandas as pd
import numpy as np
import re
import nltk
from bs4 import BeautifulSoup
from sklearn.preprocessing import LabelBinarizer
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier

First of all, you need to download the data set "tweets.csv" from the GitHub repository https://github.com/assenmacher-mat/nlp_notebooks.

__If you are running this notebook on colab (https://colab.research.google.com/), you also need to run the next chunk in order to upload the data to colab.  
Choose it in the upload window and in it will be available on colab from now on.__  
(If you are running this notebook locally on your machine, you can skip the execution of this chunk)

In [ ]:
from google.colab import files
uploaded = files.upload()

### Import the data set

__If you are running this notebook locally on your machine, you might need to adjust the path (depending on where you've saved the data).__  
(If you are running this notebook on colab, you can can leave the path unchanged)

In [2]:
imdb = pd.read_csv("imdb.csv", encoding = "utf-8")

### Next, just have a look at the data set in order to see what's inside

In [3]:
imdb.loc[:3,:]

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative


### Extract a list of texts and a list of labels from the pandas data frame

In [4]:
reviews_raw = [doc for doc in list(imdb.review)]
sentiments = [sent for sent in list(imdb.sentiment)]

### Use ``BeautifulSoup`` for cleaning the html markup

In [6]:
reviews_clean = [BeautifulSoup(rev, "html.parser").text for rev in reviews_raw]

In [7]:
reviews_clean[:3]

["One of the other reviewers has mentioned that after watching just 1 Oz episode you'll be hooked. They are right, as this is exactly what happened with me.The first thing that struck me about Oz was its brutality and unflinching scenes of violence, which set in right from the word GO. Trust me, this is not a show for the faint hearted or timid. This show pulls no punches with regards to drugs, sex or violence. Its is hardcore, in the classic use of the word.It is called OZ as that is the nickname given to the Oswald Maximum Security State Penitentary. It focuses mainly on Emerald City, an experimental section of the prison where all the cells have glass fronts and face inwards, so privacy is not high on the agenda. Em City is home to many..Aryans, Muslims, gangstas, Latinos, Christians, Italians, Irish and more....so scuffles, death stares, dodgy dealings and shady agreements are never far away.I would say the main appeal of the show is due to the fact that it goes where other shows w

### Transform the labels in the right format

In [8]:
labels=LabelBinarizer().fit_transform(sentiments)
print(labels.shape)

(50000, 1)


In [11]:
labels[:10]

array([[1],
       [1],
       [1],
       [0],
       [1],
       [1],
       [1],
       [0],
       [0],
       [1]])

### Transform the corpus to a Bag-of-words

#### (1) Define the ``Vectorizer``  
Use:
    - unigrams
    - a minimum term frequency of 10
    - counts (not binary indicators)
    - a maximum of 10.000 features

In [25]:
unigram_vectorizer = CountVectorizer(ngram_range = (1,1), 
                                     min_df = 10, 
                                     binary = False, 
                                     max_features = 10000)

### Additional question:  
If we restrict the maximum number of features, accoring to which criterion are the features omitted?  
(Hint: Look it up in the help page)

#### (2) Feed the data to the ``Vectorizer`` and create a Document-Term matrix

In [26]:
bow = unigram_vectorizer.fit_transform(reviews_clean)

#### (3) Which dimension doese the DTM have?

In [27]:
print(bow.toarray().shape)

(50000, 10000)


#### (4) Set the minimum term frequency to 100, create a new DTM and check what happens to the dimension

In [29]:
unigram_vectorizer100 = CountVectorizer(ngram_range = (1,1), min_df = 100, binary = False, max_features = 10000)
bow100 = unigram_vectorizer100.fit_transform(reviews_clean)
print(bow100.toarray().shape)

(50000, 6347)


#### (5) Create a DTM using uni-, bi- & trigrams  
(Don't restrict the maximum number of features in order to see what happens to the dimension;  
Leave the minimum term frequency at 100, you might run into memory issues otherwise;  
You will also encounter a massive impact on the runtime)

In [32]:
trigram_vectorizer = CountVectorizer(ngram_range = (1,3), min_df = 100, binary = False)
bow_tri = trigram_vectorizer.fit_transform(reviews_clean)
print(bow_tri.toarray().shape)

(50000, 20840)


#### (6) Create a DTM using uni- & bigrams while simultaneously applying TF-IDF weighting

In [33]:
tfidf = TfidfVectorizer(ngram_range = (1,2), 
                        min_df = 10, 
                        binary = False, 
                        max_features = 10000,
                        use_idf=True)

In [34]:
bow_tfidf = tfidf.fit_transform(reviews_clean)

#### (7) Inspect e.g. the entries of the intersection of the first 5 rows & the first 5 columns

In [38]:
print(bow_tfidf.toarray()[:5,:5])

[[0.         0.         0.         0.         0.        ]
 [0.         0.         0.         0.         0.        ]
 [0.         0.         0.         0.         0.        ]
 [0.         0.         0.05097438 0.         0.        ]
 [0.         0.         0.         0.         0.        ]]


### Congrats, you have finished the first day!

![](https://media.giphy.com/media/VQ77RNKX0nyaA/giphy.gif)